# Collosion and quantum-resistant lattice hasher with accumulating properties

## Shortest integer solution problem:
Given an integer `q`, a `k-times-m` matrix `M` picked uniformly at random (where `m >= k`) and a real `β`, find an integer vector `z`
such that `M*z = 0 mod q` and `||z|| <= β`

Security parameter `k` and bound `n = poly(k)`

In [20]:
k = 128
n = k^8

lmb = 1 # need to be > 0

parameter `q, mu, beta` for SIS problem

In [21]:
q = var('q')
#assume(q>n)
s = find_root(q/sqrt(log(q,2)+1) == sqrt(2)*n*k^(1/2 + lmb), 1, n^2)
q = next_prime(s)
RR(q/sqrt(ceil(log(q,2)))) >= RR(sqrt(2)*n*k^(1/2 + lmb))
print(bool(q/sqrt(ceil(log(q,2))) >= sqrt(2)*n*k^(1/2 + lmb)))
mu = ZZ(2*k*ceil(log(q,2)))
beta = RR(n*sqrt(mu))
m = ZZ(mu/2)
G = GF(q)
#print("Solution: " + str(s))
print("q: " + str(q))
print("mu: " + str(mu))
print("beta: " + str(beta))
print("m: " + str(m))

True
q: 1244142437461793964053
mu: 18176
beta: 9.71468927453313e18
m: 9088


Left and right matrices, uniform random from G

In [22]:
L = matrix([[G.random_element() for i in range(0, m)] for j in range(0,k)])
#outf = open("/tmp/matrix", "w")
#outf.write(str(L))
#outf.close()
print("done")

done


Initial hash function

In [23]:
def hash(x):
    return (L*x) % q

In [24]:
xvec = vector([G.random_element() for i in range(0, m)])
Zp_result = hash(xvec)

Because the range and domain of the current scheme differs, we need to transform our hashes back to G to keep algebraic structures and achieve accumulating properties

In [25]:
def transform(x):
    return G(int(''.join([bin(i) for i in x]).replace("0b",""),2))

In [28]:
transform(Zp_result)


815150217032424563520

The final hasher procudes a `ceil(log2(q))+1` size output from an input at most `m` size:

In [29]:
def hash(x):
    t = (L*x) % q
    return transform(t)
    

In [30]:
import hashlib
sha = hashlib.sha1()
xvec = random_vector(G, m)
sha.update(str(xvec))
print("reference sha: " + sha.hexdigest()[0:20])
print("our hash: " + hex(ZZ(hash(xvec))))

reference sha: 240dfd9fe93b9520c273
our hash: 1f8187eedd60a03e93


We want to prove that an element belongs to our 50-long array

In [59]:
import hashlib
test_data = [random_vector(G, m) for i in range(0, 5)]
target_element = test_data[2]
#print(target_element)

In [60]:
acc = vector(G, [0 for i in range(0, 128)])
for i in test_data:
    acc += L*i 

In [61]:
witness = acc + L*(target_element*-1) # we revoke our selected member from the accumulator in constant time!

In [62]:
witness + L*(target_element) == acc # checking simply works by accumulating the member to our wittness

True